In [1]:
import tensorflow as tf
import numpy as np
import time

from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
sess = tf.Session(config=config)
set_session(sess)

#nvidia-smi

Using TensorFlow backend.


In [2]:
# NOTE: load the data from the npz
dataset = np.load('./datasets/v0_eigens.npz') #contains 'train_eigens' and 'issue_eigens'

# NOTE: calculate the size of training set and validation set
#       all pre-processed features are inside 'train_eigens'
train_data_size = dataset['train_eigens'].shape[0] # train_data_size = 57159, (57159, 924)
valid_data_size = train_data_size // 5 # valid_data_size= 11431
train_data_size = train_data_size - valid_data_size # train_data_size= 45728
indices = np.arange(train_data_size + valid_data_size) # indices= [    0     1     2 ... 57156 57157 57158]

# NOTE: split dataset
train_data = dataset['train_eigens'][indices[:train_data_size]] # [:45728] 
valid_data = dataset['train_eigens'][indices[train_data_size:]] # [45728:]

# NOTE: a 896d feature vector for each user, the 28d vector in the end are
#       labels
#       896 = 32 (weeks) x 7 (days a week) x 4 (segments a day)
# train_data.shape is (45728, 924)
train_eigens = train_data[:, :-28] #(45728, 896)
train_labels = train_data[:, -28:] #(45728, 28)
valid_eigens = valid_data[:, :-28] #(11431, 896)
valid_labels = valid_data[:, -28:] #(11431, 28)
issue_eigens = dataset['issue_eigens'][:, :-28] #(37092, 896)

images = train_eigens
labels = train_labels

In [ ]:
#The size of the training input: (45728, 896)
#The size of the training label: (45728, 28)
#The size of the validation input: (11431, 896)
#The size of the validation label: (11431, 28)
#The size of the test input: (37092, 896)
#The size of the test label: (37092, 28)

### define some functions

In [3]:
# write result
import csv
import os

def write_result(name, predictions):
    if predictions is None:
        raise Exception('need predictions')

    predictions = predictions.flatten()

    if not os.path.exists('./results/'):
        os.makedirs('./results/')

    path = os.path.join('./results/', name)

    with open(path, 'wt', encoding='utf-8', newline='') as csv_target_file:
        target_writer = csv.writer(csv_target_file, lineterminator='\n')

        header = [
            'user_id',
            'time_slot_0', 'time_slot_1', 'time_slot_2', 'time_slot_3',
            'time_slot_4', 'time_slot_5', 'time_slot_6', 'time_slot_7',
            'time_slot_8', 'time_slot_9', 'time_slot_10', 'time_slot_11',
            'time_slot_12', 'time_slot_13', 'time_slot_14', 'time_slot_15',
            'time_slot_16', 'time_slot_17', 'time_slot_18', 'time_slot_19',
            'time_slot_20', 'time_slot_21', 'time_slot_22', 'time_slot_23',
            'time_slot_24', 'time_slot_25', 'time_slot_26', 'time_slot_27',
        ]

        target_writer.writerow(header)

        for i in range(0, len(predictions), 28):
            # NOTE: 57159 is the offset of user ids
            userid = [57159 + i // 28]
            labels = predictions[i:i+28].tolist()

            target_writer.writerow(userid + labels)

# define metric AUC
import sklearn.metrics

def auc(guess, truth):
    """
    """
    guess = guess.flatten()
    truth = truth.flatten()
    
    fprs, tprs, _ = sklearn.metrics.roc_curve(truth, guess)

    return sklearn.metrics.auc(fprs, tprs)

In [4]:
from keras.models import Sequential
from keras.layers import Conv1D, Dense, Activation, Flatten, Bidirectional, LSTM, MaxPool1D, GRU
from keras.layers.core import Reshape

In [39]:
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)


def build_model():
    model = Sequential()
    model.add(Dense(28, activation='relu'))
    model.add(Reshape((28,1)))
    
    model.add(Conv1D(kernel_size = (11), filters = 5, activation='relu'))
    model.add(MaxPool1D(pool_size = (1), strides=(1)))    
    model.add(Bidirectional(LSTM(5, return_sequences=True)))   
    #model.add(GRU(28, return_sequences = True))
    #model.add(GRU(32, return_sequences = True))      
    model.add(Flatten())
    
    model.add(Dense(28, activation='sigmoid'))
    model.compile(optimizer='rmsprop', loss='binary_crossentropy')

    return model

model = build_model()

start = time.time()

images_X = np.expand_dims(images, axis=2)
valid_eigens_X = np.expand_dims(valid_eigens, axis=2)

model.fit(
    images, 
    labels, 
    epochs=11, 
    batch_size=450,
    validation_data=(valid_eigens, valid_labels),
    shuffle=True)

print("\nTime elapsed: {} seconds\n".format(time.time() - start))

valid_predict= model.predict(valid_eigens)
score = auc(
    valid_predict, 
    valid_labels
)

print(score)

Train on 45728 samples, validate on 11431 samples
Epoch 1/11
45728/45728 [==============================] - 15s 334us/step - loss: 0.3248 - val_loss: 0.2370
Epoch 2/11
45728/45728 [==============================] - 5s 109us/step - loss: 0.2300 - val_loss: 0.2245
Epoch 3/11
45728/45728 [==============================] - 5s 107us/step - loss: 0.2221 - val_loss: 0.2198
Epoch 4/11
45728/45728 [==============================] - 5s 108us/step - loss: 0.2168 - val_loss: 0.2149
Epoch 5/11
45728/45728 [==============================] - 5s 108us/step - loss: 0.2119 - val_loss: 0.2114
Epoch 6/11
45728/45728 [==============================] - 5s 106us/step - loss: 0.2085 - val_loss: 0.2086
Epoch 7/11
45728/45728 [==============================] - 5s 108us/step - loss: 0.2064 - val_loss: 0.2075
Epoch 8/11
45728/45728 [==============================] - 5s 108us/step - loss: 0.2050 - val_loss: 0.2073
Epoch 9/11
45728/45728 [==============================] - 5s 108us/step - loss: 0.2039 - val_loss: 0.

In [ ]:
0.8838898227839855 #kaggle 0.87968
    model = Sequential()
    model.add(Dense(28, activation='relu'))
    model.add(Reshape((28,1)))
    
    model.add(Conv1D(kernel_size = (9), filters = 5, input_shape=(896, 1), activation='relu'))
    model.add(MaxPool1D(pool_size = (1), strides=(1)))
    model.add(Bidirectional(LSTM(5, return_sequences=True)))     
    #model.add(GRU(28, return_sequences = True))
    #model.add(GRU(32, return_sequences = True))      
    model.add(Flatten())
    
    model.add(Dense(28, activation='sigmoid'))
    model.compile(optimizer='rmsprop', loss='binary_crossentropy')

0.8843608681364775 # kaggle 0.88043
    model.add(Conv1D(kernel_size = (11), filters = 5, input_shape=(896, 1), activation='relu'))

In [22]:
start = time.time()

issue_eigens_X = np.expand_dims(issue_eigens, axis=2)
issue_guesss = model.predict(issue_eigens)
write_result('tf101_model_d.csv', issue_guesss)

print("\nTime elasped: {} seconds\n".format(time.time() - start))


Time elasped: 16.965404987335205 seconds



In [40]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_70 (Dense)             (None, 28)                25116     
_________________________________________________________________
reshape_27 (Reshape)         (None, 28, 1)             0         
_________________________________________________________________
conv1d_26 (Conv1D)           (None, 18, 5)             60        
_________________________________________________________________
max_pooling1d_26 (MaxPooling (None, 18, 5)             0         
_________________________________________________________________
bidirectional_25 (Bidirectio (None, 18, 10)            440       
_________________________________________________________________
flatten_28 (Flatten)         (None, 180)               0         
_________________________________________________________________
dense_71 (Dense)             (None, 28)                5068      
Total para

In [37]:
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)


def build_model():
    model = Sequential()
    model.add(Dense(28, activation='elu'))
    model.add(Dense(28, activation='elu'))
    model.add(Dense(28, activation='relu'))
    model.add(Dense(28, activation='relu'))
    model.add(Dense(28, activation='sigmoid'))
    model.compile(optimizer='rmsprop', loss='binary_crossentropy')

    return model

model = build_model()

start = time.time()

images_X = np.expand_dims(images, axis=2)
valid_eigens_X = np.expand_dims(valid_eigens, axis=2)

model.fit(
    images, 
    labels, 
    epochs=11, 
    batch_size=450,
    validation_data=(valid_eigens, valid_labels),
    shuffle=True)

print("\nTime elapsed: {} seconds\n".format(time.time() - start))

valid_predict= model.predict(valid_eigens)
score = auc(
    valid_predict, 
    valid_labels
)

print(score)

Train on 45728 samples, validate on 11431 samples
Epoch 1/11
45728/45728 [==============================] - 10s 220us/step - loss: 0.3729 - val_loss: 0.2363
Epoch 2/11
45728/45728 [==============================] - 1s 13us/step - loss: 0.2257 - val_loss: 0.2207
Epoch 3/11
45728/45728 [==============================] - 1s 13us/step - loss: 0.2185 - val_loss: 0.2191
Epoch 4/11
45728/45728 [==============================] - 1s 13us/step - loss: 0.2152 - val_loss: 0.2152
Epoch 5/11
45728/45728 [==============================] - 1s 12us/step - loss: 0.2125 - val_loss: 0.2138
Epoch 6/11
45728/45728 [==============================] - 1s 12us/step - loss: 0.2100 - val_loss: 0.2111
Epoch 7/11
45728/45728 [==============================] - 1s 12us/step - loss: 0.2074 - val_loss: 0.2097
Epoch 8/11
45728/45728 [==============================] - 1s 12us/step - loss: 0.2056 - val_loss: 0.2138
Epoch 9/11
45728/45728 [==============================] - 1s 12us/step - loss: 0.2041 - val_loss: 0.2086
Epo

In [38]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_65 (Dense)             (None, 28)                25116     
_________________________________________________________________
dense_66 (Dense)             (None, 28)                812       
_________________________________________________________________
dense_67 (Dense)             (None, 28)                812       
_________________________________________________________________
dense_68 (Dense)             (None, 28)                812       
_________________________________________________________________
dense_69 (Dense)             (None, 28)                812       
Total params: 28,364
Trainable params: 28,364
Non-trainable params: 0
_________________________________________________________________
None
